In [14]:
#Import Packages
import pandas as pd
import numpy as np
import math
import warnings
import ast
import random as rand
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

## Prepare Data

In [15]:
def Process_External_Data(indeed_df):
    #External Data Process
    indeed_df=indeed_df.rename(columns={'name':'company name','happiness':'scores'})
    for i in range(len(indeed_df)):
        if indeed_df['scores'][i]=='{}':
            continue
        else:   
            for k,v in ast.literal_eval(indeed_df['scores'][i]).items():
                indeed_df[k]= ''
            break

    for i in range(len(indeed_df)):
        try:
            for k,v in ast.literal_eval(indeed_df['scores'][i]).items():
                indeed_df[k][i]= v
        except:
            continue

    for k,v in ast.literal_eval(indeed_df['ratings'][0]).items():
        indeed_df[k]= ''

    for i in range(len(indeed_df)):
        try:
            for k,v in ast.literal_eval(indeed_df['ratings'][i]).items():
                indeed_df[k][i]= v
        except:
            continue
    indeed_df=indeed_df.drop(columns=['ratings','scores'])
    indeed_df=indeed_df.reset_index()
    indeed_df=indeed_df.drop(columns=['index'])  
    for i in range(len(indeed_df)):
        if indeed_df['Work Happiness Score'][i]=='':
            indeed_df['Work Happiness Score'][i]=np.NaN
    for i in range(len(indeed_df)):
        if indeed_df['Inclusion'][i]=='':
            indeed_df['Inclusion'][i]=np.NaN

    indeed_df['num_locations']=''
    for i in range(len(indeed_df)):
        if len(ast.literal_eval(indeed_df['locations'][i]))==0:
            indeed_df['num_locations'][i]=1
        else:
            indeed_df['num_locations'][i]=len(ast.literal_eval(indeed_df['locations'][i]))

    indeed_df['employees_s']=''
    for i in range(len(indeed_df)):
        if indeed_df['employees'][i]=='2 to 10':
            indeed_df['employees_s'][i]=1
        elif indeed_df['employees'][i]=='11 to 50':
            indeed_df['employees_s'][i]=2
        elif indeed_df['employees'][i]=='51 to 200':
            indeed_df['employees_s'][i]=3
        elif indeed_df['employees'][i]=='201 to 500':
            indeed_df['employees_s'][i]=4
        elif indeed_df['employees'][i]=='501 to 1,000':
            indeed_df['employees_s'][i]=5
        elif indeed_df['employees'][i]=='1,001 to 5,000':
            indeed_df['employees_s'][i]=6
        elif indeed_df['employees'][i]=='5,001 to 10,000':
            indeed_df['employees_s'][i]=7
        elif indeed_df['employees'][i]=='10,000+':
            indeed_df['employees_s'][i]=8
        else:
            indeed_df['employees_s'][i]=np.random.randint(0,9)
    indeed_df['revenue_s']=''
    for i in range(len(indeed_df)):
        if indeed_df['revenue'][i]=='less than $1M (USD)':
            indeed_df['revenue_s'][i]=1
        elif indeed_df['revenue'][i]=='$1M to $5M (USD)':
            indeed_df['revenue_s'][i]=2
        elif indeed_df['revenue'][i]=='$5M to $25M (USD)':
            indeed_df['revenue_s'][i]=3
        elif indeed_df['revenue'][i]=='$25M to $100M (USD)':
            indeed_df['revenue_s'][i]=4
        elif indeed_df['revenue'][i]=='$100M to $500M (USD)':
            indeed_df['revenue_s'][i]=5
        elif indeed_df['revenue'][i]=='$500M to $1B (USD)':
            indeed_df['revenue_s'][i]=6
        elif indeed_df['revenue'][i]=='$1B to $5B (USD)':
            indeed_df['revenue_s'][i]=7
        elif indeed_df['revenue'][i]=='$5B to $10B (USD)':
            indeed_df['revenue_s'][i]=8
        elif indeed_df['revenue'][i]=='more than $10B (USD)':
            indeed_df['revenue_s'][i]=9
        else:
            indeed_df['revenue_s'][i]=np.random.randint(0,10)
    indeed_df['business age']=2022-indeed_df['year']
    indeed_df=indeed_df[['company name','business age','rating','ceo_approval','industry',
                         'Work Happiness Score','Inclusion','num_locations','employees_s','revenue_s']]
    indeed_df=indeed_df.dropna()
    indeed_df['ceo_approval']=indeed_df['ceo_approval'].str.strip('%').astype(int)
    indeed_df=indeed_df.rename(columns={'company name':'company','employees_s':'number of employee',
                                       'revenue_s':'revenue','num_locations':'office locations',
                                       'rating':'company rating','Work Happiness Score':'employee satisfaction',
                                       'Inclusion':'employee diversity','ceo_approval':'ceo approval',})
    indeed_df['employee satisfaction']=indeed_df['employee satisfaction'].astype(int)
    indeed_df['employee diversity']=indeed_df['employee diversity'].astype(int)
    indeed_df['office locations']=indeed_df['office locations'].astype(int)
    indeed_df['number of employee']=indeed_df['number of employee'].astype(int)
    indeed_df['revenue']=indeed_df['revenue'].astype(int)

    indeed_df['listed company indicator']=np.random.randint(0,2, indeed_df.shape[0])
    indeed_df['website complexity']=np.random.randint(0,100, indeed_df.shape[0])
    indeed_df['social media popularity']=np.random.randint(0,5000000, indeed_df.shape[0])
    indeed_df['social media presence']=np.random.randint(0,10, indeed_df.shape[0])
    indeed_df['social mention']=np.random.randint(0,1000, indeed_df.shape[0])
    indeed_df['community practice']=np.random.randint(0,10, indeed_df.shape[0])
    indeed_df['carbon footprint']=np.random.randint(0,5000000, indeed_df.shape[0])
    indeed_df['environmental practice']=np.random.randint(0,10, indeed_df.shape[0])
    
    return indeed_df

def Process_Internal_Data(df_company, df_company_employee, df_users_rank,df_organization):
    # Get Internal 
    company_uid_list=[]
    num_individ_from_company_list = []
    sum_individ_impact_score_list = []
    mean_individ_impact_score_list = []
    number_of_upload_list = []
    sum_of_the_number_of_followers_list = []

    for i in range(len(df_company)):
        # get empolyees' individual impact score
        company_uid_list.append(df_company['uid'][i]) 
        index=df_company_employee[df_company_employee['uid']==df_company['uid'][i]].index[0]
        member_info = df_users_rank.loc[df_users_rank['uid'].isin(ast.literal_eval(df_company_employee['employee_list'][index]))]
        num_individ_from_company_list.append(member_info['uid'].nunique())
        sum_individ_impact_score_list.append(round(member_info['score'].sum()))
        mean_individ_impact_score_list.append(round(member_info['score'].sum()/member_info['uid'].nunique(),2))
        number_of_upload_list.append(len(df_organization[df_organization['uid']==df_company['uid'][i]]))


    # cast lists of metrics to internal_data dataframe 
    df_company_add = pd.DataFrame({'uid': company_uid_list,
                                     'number of individual from the company': num_individ_from_company_list, 
                                     'individuals impact scores_sum': sum_individ_impact_score_list, 
                                     'individuals impact scores_mean': mean_individ_impact_score_list,
                                     'number of upload': number_of_upload_list}).fillna(60)
    df_internal=df_company.merge(df_company_add,how='left',on='uid')

    # calculate the basic mean or proportion date
    df_internal['number of views_mean']=round(df_internal['number of views_sum']/(df_internal['number of upload']+1),2)
    df_internal['rating score_mean']=round(df_internal['rating score_sum']/(df_internal['number of upload']+1),2)
    df_internal['number of comment_mean']=round(df_internal['number of comment_sum']/(df_internal['number of upload']+1),2)
    
    return df_internal

def Merge_Data(df_external, df_internal):
    #merge internal and external
    df_organ_all = pd.merge(left=df_internal, right=df_external, on='company',how='inner')
    df_organ_all['proportion of the company employees involved']=round(df_organ_all['number of individual from the company']/df_organ_all['number of employee'],3)
    return df_organ_all

## Calculate Score

In [16]:
def Calculate_Score(df_weights, df_cat_weights, df_organ_all):
    #Transform and Scale
    for i in range(len(df_weights)):
        k=df_weights['k'][i]
        u=df_weights['Upper Bound'][i]
        if df_weights['Transformation'][i]=='Log':
            df_organ_all[df_weights['Metrics'][i]]=df_organ_all[df_weights['Metrics'][i]].apply(lambda x: 200/(1+math.exp(-k*x))-100 if x<=u else 100)
        elif df_weights['Transformation'][i]=='Linear':
            df_organ_all[df_weights['Metrics'][i]]=df_organ_all[df_weights['Metrics'][i]].apply(lambda x: (100*x/u) if (100*x/u) < 100 else 100)
        elif df_weights['Transformation'][i]=='Boolean':
            df_organ_all[df_weights['Metrics'][i]]=df_organ_all[df_weights['Metrics'][i]]*100
        else:
            continue

    df_organ_all['number of employee']=df_organ_all['number of employee'].apply(lambda x : x*20 if x <= 5 else 100)
    df_organ_all['revenue']=df_organ_all['revenue'].apply(lambda x : x*20 if x <= 5 else 100)
    df_organ_all['office locations']=df_organ_all['office locations'].apply(lambda x : 50+(x-1)*20 if x <= 2 else (70+(x-2)*10 if x <=5 else 100))

    for i in range(len(df_weights)):
        c=df_weights['Metrics'][i]
        d=df_weights['Dimension_weight'][i]
        df_organ_all[c]=df_organ_all[c]*d

    for c in df_weights['Category'].unique():
        df_organ_all[c]=0   
    for c in df_weights['Category'].unique():
        for i in range(len(df_weights)):
            if df_weights['Category'][i]==c:
                df_organ_all[c]+=df_organ_all[df_weights['Metrics'][i]]
    col_list=list(df_weights['Category'].unique())
    col_list.insert(0,'company')
    df_organ_all=df_organ_all[col_list]
    df_organ_all['total_score']=0
    for i in range(len(df_cat_weights)):
        c=df_cat_weights['Category'][i]
        d=df_weights['Category_weight'][i]
        df_organ_all[c]=df_organ_all[c]*d
        df_organ_all['total_score']+=df_organ_all[c]
    df_organ_all['rank']=df_organ_all['total_score'].rank(method='min',ascending=False).astype(int)
    
    return df_organ_all

if __name__ == "__main__":
    df_weights=pd.read_csv('input_data/weights.csv')
    df_cat_weights=pd.read_csv('input_data/category_weights.csv')
    df_users_rank=pd.read_csv('input_data/df_users_rank.csv')
    df_company=pd.read_csv('input_data/df_company.csv')
    df_company_employee=pd.read_csv('input_data/df_company_employee.csv')
    indeed_df=pd.read_csv('input_data/indeed_company_data.csv')
    df_organization=pd.read_csv('input_data/df_organization.csv')
    
    df_external_data = Process_External_Data(indeed_df)
    df_internal_data = Process_Internal_Data(df_company, df_company_employee, df_users_rank,df_organization)
    df_organ_all = Merge_Data(df_external_data, df_internal_data)
    df_organ_impact_score = Calculate_Score(df_weights, df_cat_weights, df_organ_all)

df_organ_impact_score

,company,Company Capacity,Media Coverage,Public Opinions,social and environmental impact,employee's individual impact score,organization's impact score,organization and employee interaction,total_score,rank
0,Indiana Packers Corporation,12.198949,12.912295,6.0396,4.396000,3.228575,26.280196,9.839746,74.895360,7
1,The Container Store,12.452444,12.823306,6.3924,12.813725,2.965068,28.998283,9.839746,86.284971,1
2,DEFENDERS,14.000000,10.619046,5.3088,12.383488,3.057678,24.257033,9.948580,79.574624,3
3,Amerita,11.200000,11.479772,5.5860,11.724343,1.437784,29.550080,9.960420,80.938399,2
4,"Lithia Motors, Inc.",13.870397,9.170000,5.2136,9.616646,2.329517,27.585601,9.936644,77.722405,4
5,Apartment Management Consultants,10.712539,11.786240,5.8492,11.804050,2.663344,24.682184,9.904037,77.401593,5
6,Swire Coca-Cola,12.600000,9.719778,4.9140,12.544000,3.057678,22.627361,9.977794,75.440611,6
7,Akumin,10.181675,9.765000,4.2252,12.383488,3.124872,24.641917,9.962442,74.284593,8
